In [ ]:
# 조선 뉴스 라이브러리100(~19991231) 목록, 본문

import os
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd

# 브라우저 옵션 설정
chrome_options = Options()
# chrome_options.add_argument("--headless")  # 브라우저 창을 띄우지 않음
chrome_options.add_argument("--disable-gpu")  # GPU 비활성화
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

# 크롬 브라우저 실행 (최신 Selenium은 ChromeDriver 설치 필요 없음)
driver = webdriver.Chrome(options=chrome_options)


def login_kakao(username, password):
    """카카오 로그인 페이지로 직접 접근하여 로그인합니다."""
    try:
        login_url = "https://www.chosun.com/subscribe/signin/"
        driver.get(login_url)

        # 아이디 입력 필드가 나타날 때까지 대기하고 입력
        username_field = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "username"))
        )
        username_field.send_keys(username)

        # 비밀번호 입력 필드가 나타날 때까지 대기하고 입력
        password_field = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "subsPassword"))
        )
        password_field.send_keys(password)

        # 로그인 버튼 클릭
        login_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.ID, "subsSignIn"))
        )
        login_button.click()
        time.sleep(3)
        print("로그인 버튼 클릭 성공!")

    except Exception as e:
        print(f"로그인 중 오류 발생: {e}")


def fetch_articles(search_query, start_date, end_date):
    """특정 검색어에 대해 기사를 검색하고 정보를 수집."""
    article_list = []
    page_number = 0
    sort_order = 1
    base_url = "https://newslibrary.chosun.com/search/search_result.html"

    while True:
        # 검색 URL 생성
        search_url = (
            f"{base_url}?case_num=1&sort={sort_order}&page={page_number}&size=10"
            f"&query={search_query}&date=date_select&ds={start_date}&de={end_date}&field=&type=&wrt=&set_date="
        )

        # 검색 페이지 열기
        driver.get(search_url)

        # 기사 목록이 로드될 때까지 대기
        try:
            WebDriverWait(driver, 5).until(
                EC.presence_of_element_located((By.CLASS_NAME, "search_news"))
            )
        except:
            print(f"{search_query} - 더 이상 페이지가 없습니다.")
            break

        print(f"{search_query} - {page_number} 페이지를 처리 중입니다...")
        
        # HTML 파싱
        soup = BeautifulSoup(driver.page_source, "html.parser")

        # 기사 목록 추출
        articles = soup.find_all("dl", class_="search_news")
        if not articles:
            break  # 더 이상 기사가 없으면 종료

        # 기사 정보 추출
        for article in articles:
            title_tag = article.find("dt").find("a")
            if title_tag:
                title = title_tag.text.strip()
                url = "https://newslibrary.chosun.com" + title_tag["href"]

                # 날짜, 면수, 섹션, 저자 정보 추출
                art_info_tag = article.find("dd", class_="art_info")
                date = art_info_tag.contents[0].strip() if art_info_tag.contents else ""
                spans = art_info_tag.find_all("span")
                page = spans[0].text.strip() if len(spans) > 0 else ""
                section = spans[1].text.strip() if len(spans) > 1 else ""
                writer = spans[2].text.strip() if len(spans) > 2 else ""

                # 본문 수집
                content = clean_content(url)

                # 기사 정보 리스트에 추가
                article_list.append({
                    "media": '조선일보',
                    "title": title,
                    "date": date,
                    "page": page,
                    "section": section,
                    'article_type': '',
                    "writerr": writer,
                    "url": url,
                    "content": content,
                })

        # 다음 페이지로 이동
        page_number += 1

    return article_list

def clean_content(url):
    try:
        driver.get(url)
        WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.ID, "subtitle1"))
        )
        time.sleep(1)

        soup = BeautifulSoup(driver.page_source, "html.parser")
        tag = soup.find("p", id="subtitle1")

        if tag:
            # 불필요한 태그 제거 후 텍스트 추출
            for t in tag.find_all(["span", "img", "a"]):
                t.decompose()
            return tag.get_text(separator="\n", strip=True)

    except Exception as e:
        print(f"본문 추출 실패: {e}")

    return ""  # 본문이 없을 경우 빈 문자열 반환    

def save_to_excel(data, filename):
    """수집한 데이터를 엑셀 파일로 저장."""
    df = pd.DataFrame(data)
    df.drop_duplicates(subset=['url'], keep='first', inplace=True)
    df.to_excel(filename, index=False)

# 아래에서 사용자 입력 설정
if __name__ == "__main__":
    # 사용자 정보 및 검색어 설정
    username = "shincha23@naver.com"
    password = "sje126604!"
    search_queries = ["검색어색어"]
    start_date = "19610101"
    end_date = "19611231"

    # 조선일보 로그인
    login_kakao(username, password)

    # 모든 검색어에 대해 기사 수집
    all_articles = []
    for query in search_queries:
        articles = fetch_articles(query, start_date, end_date)
        all_articles.extend(articles)    

    # 결과를 엑셀 파일로 저장
    save_to_excel(all_articles, f"조선일보 라이브러리 {start_date}~{end_date}.xlsx")

    # 드라이버 종료
    driver.quit()

    print("스크래핑이 완료되었습니다!")


In [ ]:
# 뉴스탭 기사 목록(19930101부터 가능)

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time
import urllib.parse

# 크롬 드라이버 옵션 설정
chrome_options = Options()
chrome_options.add_argument("--headless")  # 필요시 활성화
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--start-maximized")
chrome_options.add_argument("--disable-extensions")  # 확장 프로그램 비활성화
chrome_options.add_argument("--disable-gpu")  # GPU 가속 비활성화

# 키워드와 날짜 범위 설정
keywords = [
    "검색어",
]
start_date = "19930101"
end_date = "20240851"


seen_urls = set()  # 중복된 URL을 저장하기 위한 집합
# 웹 드라이버 초기화
driver = webdriver.Chrome(options=chrome_options)

def fetch_search_results(keyword, start_date, end_date):
    """키워드와 날짜 범위에 따라 기사를 스크래핑합니다."""
    base_url = "https://www.chosun.com/nsearch/?query="
    page = 1  # 페이지 시작 번호 설정
    results = []
    previous_articles = set()  # 이전 페이지의 기사 URL 또는 제목을 저장할 집합

    while True:
        print(f"{keyword} 페이지 {page}")
        search_url = (
            f"{base_url}{urllib.parse.quote(keyword)}&"
            f"page={page}&siteid=www&sort=1&date_period=&date_start={start_date}&date_end={end_date}&"
            "writer=&field=&emd_word=&expt_word=&opt_chk=false&app_check=0&website=www,chosun&category="
        )
        driver.get(search_url)
        time.sleep(2)

        try:
            # 기사 목록 가져오기
            search_feed_element = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CLASS_NAME, "search-feed"))
            )
            search_feed_html = search_feed_element.get_attribute("outerHTML")
            search_feed = BeautifulSoup(search_feed_html, 'html.parser')
            articles = search_feed.find_all("div", class_="story-card-wrapper") if search_feed else []

            print(f"찾은 기사 수: {len(articles)}")  # 디버깅용 출력

            if not articles:
                print(f"{keyword} - 더 이상 기사가 없습니다.")
                break  # 더 이상 기사가 없으면 루프 종료

            current_articles = set()  # 현재 페이지의 기사 URL 또는 제목을 저장할 집합

            # 기사 정보 추출
            for article in articles:
                try:
                    title_tag = article.find("a", class_="text__link story-card__headline | box--margin-none text--black font--primary h3 text--left")
                    title = title_tag.text.strip() if title_tag else ""
                    url = title_tag["href"] if title_tag else ""         

                    # 기사 식별자를 URL 또는 제목으로 설정
                    article_identifier = url  # 또는 title

                    current_articles.add(article_identifier)

                    span_elements = article.find_all("span")
                    date = span_elements[-1].text.strip() if span_elements else ""

                    section_element = article.find("span", class_="hover-underline | box--pointer")
                    section = section_element.text.strip() if section_element else ""

                    writer_element = article.find("span", class_="box--hidden-sm")
                    writer = writer_element.text.strip().replace('|', '').strip() if writer_element else ""

                    # 결과 리스트에 추가
                    results.append({
                        'media': '조선일보',
                        'title': title,
                        'date': date,
                        'section': section,
                        'article_type': '',
                        'writer': writer,
                        'page': '',
                        'url': url,
                        'content': ''  # 컨텐츠 정보 없음
                    })
                except Exception as e:
                        print(f"기사 추가됨: {title}, {url}")  # 디버그 메시지

            # 이전 페이지의 기사와 현재 페이지의 기사를 비교하여 동일하면 루프 종료
            if previous_articles == current_articles and len(current_articles) == 10:
                print(f"{keyword} - 이전 페이지와 동일한 10개의 기사가 발견되어 루프를 종료합니다.")
                break
            else:
                previous_articles = current_articles  # 이전 페이지의 기사를 현재 페이지의 기사로 업데이트
                page += 1  # 다음 페이지로 이동

            # 만약 현재 페이지의 기사가 10개 미만이면 루프 종료 (필요 시 주석 해제)
            if len(current_articles) < 10:
                print(f"{keyword} - 마지막 페이지에 도달했습니다.")
                break

        except Exception as e:
            print(f"페이지 로딩 오류 발생: {e}")
            break  # 오류 발생 시 루프 종료

    return results  # 모든 페이지를 탐색한 후 최종 결과 반환


# 엑셀 저장 함수
def save_to_excel(data, filename):
    df = pd.DataFrame(data)
    df = df[~df['title'].str.contains('블락비|지코|학폭|뇌섹남|나인뮤지스', case=False, na=False)]
    df = df.drop_duplicates(subset=['url'], keep='first')
    df.to_excel(filename, index=False)


# 모든 키워드에 대해 검색 수행
all_results = []
for keyword in keywords:
    # 매개변수 전달
    results = fetch_search_results(keyword, start_date, end_date)
    all_results.extend(results)

# 결과를 엑셀로 저장
save_to_excel(all_results, f'1조선일보 {start_date}~{end_date}.xlsx')

# 드라이버 종료
driver.quit()


In [ ]:
#본문 기사 저장하기

import os
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

# 1. 출처와 파일명 변수 설정
file_name = '파일명'  # 파일명 설정
base_path = rf'경로'  # 경로 설정

# 2. 임시 파일 경로와 최종 파일 경로 생성
temp_output_path = os.path.join(base_path, f'{file_name}.xlsx')
# 임시 파일 에러를 대비해 본 파일에 덮어쓴다.
output_path = os.path.join(base_path, f'{file_name}_본문.xlsx')  # 최종 파일 경로
error_log_path = os.path.join(base_path, 'error_log.txt')  # 오류 로그 경로

# 3. 파일 경로 생성 및 엑셀 파일 불러오기
input_file = os.path.join(base_path, f'{file_name}.xlsx')
df = pd.read_excel(input_file, engine='openpyxl')

# Selenium 헤드리스 설정
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

# Chrome 브라우저 실행
driver = webdriver.Chrome(options=chrome_options)

# URL에서 본문을 스크랩하는 함수 정의
def get_content_from_url(url):
    try:
        driver.get(url)

        if "원하시는 페이지를 찾을 수 없습니다" in driver.page_source:
            print(f"Page not found: {url}")
            return None, None

        # CSS 선택자 리스트
        possible_selectors = [
            '.article-body', '#article-body', '.par', '#par',
            '.read-news.article', '#read-news.article', '.sticky-article', '#sticky-article',
            '.articleBody', '#articleBody', '.contents', '#contents',
            '.news_body', '#news_body', '.Paragraph', '#Paragraph',
            '.article_body', '#article_body', '.content', '#content',
            '.article-view-content-div', '#article-view-content-div'
        ]

        soup = BeautifulSoup(driver.page_source, 'html.parser')
        all_content = []  # 본문 내용을 저장할 리스트

        for selector in possible_selectors:
            elements = soup.select(selector)
            for element in elements:
                for caption in element.find_all(
                    ['table', 'div', 'dl', 'figure'],
                    class_=['content_popup_ad', 'right_img', 'photo-layout', 'news_imgbox']
                ):
                    caption.decompose()

                for caption in element.find_all('div', style=''):
                    caption.decompose()

                for caption in element.find_all('figcaption'):
                    caption.decompose()                    

                ad = element.find('div', class_='ad-area')
                if ad:
                    next_sibling = ad.find_next_sibling()
                    while next_sibling:
                        next_sibling.decompose()
                        next_sibling = ad.find_next_sibling()
                    ad.decompose()

                                # 서브타이틀 추출
                subtitle = ""
                subtitle_element = soup.find("p", class_="font--primary font--size-md-20 font--size-sm-18 text--black")
                if subtitle_element:
                    subtitle = subtitle_element.find("span").get_text(separator="\n", strip=True) if subtitle_element else ""
                    subtitle_element.extract()  # 서브 타이틀 제거         

                all_content.append(element.get_text(separator='\n', strip=True))

        content = '\n\n'.join(all_content) if all_content else ""
        return content, subtitle

    except Exception as e:
        print(f"Error fetching URL {url}: {e}")
        return None, None

# 스크래핑 진행 상태와 오류 로그 관리
error_log = []
total_articles = len(df)

# 각 행의 URL에서 본문 내용을 스크랩하여 'content' 열에 저장합니다.
for index, row in df.iterrows():
    try:
        if pd.notnull(row.get('content')) and row['content'].strip():
            print(f"Skipping {index + 1}/{total_articles}: {row['url']} (content already exists)")
            continue
    except AttributeError:
        pass

    print(f"Processing {index + 1}/{total_articles}: {row['url']}")
    content, subtitle = get_content_from_url(row['url'])

    if content is None:
        error_log.append(row['url'])
        print(f"Skipping {row['url']} due to missing content.")
        continue

    if content.startswith('='):
        content = "'" + content

    df.at[index, 'content'] = content
    df.at[index, 'sub-title'] = subtitle

    # 100개마다 임시 파일로 저장
    if (index + 1) % 100 == 0:
        df.to_excel(temp_output_path, index=False)
        print(f"임시 파일을 저장했습니다: {temp_output_path}")

# 최종 엑셀 파일 저장
with pd.ExcelWriter(output_path, engine='xlsxwriter') as writer:
    workbook = writer.book
    text_format = workbook.add_format({'num_format': '@'})

    df = df[~df['title'].str.contains('블락비|나인뮤지스', case=False, na=False)]
    df = df[~df['content'].str.contains('블락비|나인뮤지스', case=False, na=False)]
    df.to_excel(writer, index=False, sheet_name='Sheet1')

    worksheet = writer.sheets['Sheet1']
    worksheet.set_column(8, 8, None, text_format)

# 웹드라이버 종료
driver.quit()

# 오류 로그 저장
if error_log:
    with open(error_log_path, 'w') as f:
        for url in error_log:
            f.write(f"{url}\n")

print("본문 내용을 성공적으로 스크랩하여 저장했습니다.")



